http://web.mta.info/developers/turnstile.html
https://groups.google.com/forum/#!searchin/mtadeveloperresources/turnstile/mtadeveloperresources/tynsUKSJtjk/BIdNU_kdKnwJ

C/A      = Control Area (A002)
UNIT     = Remote Unit for a station (R051)
SCP      = Subunit Channel Position represents an specific address for a device
           (02-00-00)
STATION  = Represents the station name the device is located at
LINENAME = Represents all train lines that can be boarded at this station
           Normally lines are represented by one character.
           LINENAME 456NQR repersents train server for
           4, 5, 6, N, Q, and R trains.

DIVISION = Represents the Line originally the station belonged to BMT, IRT,
           or IND
DATE     = Represents the date (MM-DD-YY)
TIME     = Represents the time (hh:mm:ss) for a scheduled audit event
DESc     = Represent the "REGULAR" scheduled audit event
           (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning,
           or troubleshooting activities.
           2. Additionally, there may be a "RECOVR AUD" entry:
           This refers to a missed audit that was recovered.

ENTRIES  = The cumulative entry register value for a device
EXIST    = The cumulative exit register value for a device

In [102]:
import pandas as pd
import dateutil.parser

def date_year_week(date_str):
    parsed_date = dateutil.parser.parse(date_str).isocalendar()
    return '{}-{}'.format(parsed_date[0], parsed_date[1])
    
def calculate_entry_and_exit(list_date_entry_exit):
    """
    each item in `list_date_entry` is another list [date, entry, exit]
    """
    # first sort by entry ascending
    list_date_entry_exit.sort(key=lambda x: x[1])
    lowest_entry = list_date_entry_exit[0][1]
    highest_entry = list_date_entry_exit[-1][1]
    
    # now sort by entry descending
    list_date_entry_exit.sort(key=lambda x: x[1])
    lowest_exit = list_date_entry_exit[0][2]
    highest_exit = list_date_entry_exit[-1][2]
    #print(list_date_entry)
    total_traffic = (highest_entry - lowest_entry) + (highest_exit - lowest_exit)
    return total_traffic

# Calculate counts for each turnstile per week

columns = ['control_area', 'unit', 'scp', 'station', 'line_name',
            'division', 'date', 'time', 'desc', 'entries', 'exits']
df = pd.read_csv(
        'turnstile_170624.txt'
       #'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170624.txt'
)
df.columns = columns

df = df[(df.station == '1 AV') | (df.station == '103 ST') 
        | (df.station == '103 ST-CORONA') 
        | (df.station == '104 ST') | (df.station == '110 ST')
        | (df.station == '111 ST')
        | (df.station == '116 ST')]

df['value_date_entry_exit'] = df.apply(lambda row: [dateutil.parser.parse(row.date  + ' ' + row.time), row.entries, row.exits], axis=1)
df['index_day'] = df.apply(lambda row: (row.control_area, row.unit, row.scp, row.station, dateutil.parser.parse(row.date)), axis=1)

df_group_day = df.groupby('index_day')['value_date_entry_exit'].apply(list)


In [98]:
# Currently a series, convert to a dataframe
df_group_day = pd.DataFrame({'date_and_entries': df_group_day})
df_group_day['entries_and_exits_total'] = df_group_day['date_and_entries'].map(calculate_entry_and_exit)


In [99]:
df_group_day['station'] = df_group_day.apply(lambda row: row.name[3], axis=1)


In [100]:
df_group_week_series = df_group_day.groupby('station').sum()
# df_group_week_series = df_group_week_series.sort_values('station')
df_group_week_series
# 1621136548

,entries_and_exits_total
station,
1 AV,1788365727
103 ST,295320
103 ST-CORONA,189833
104 ST,21469
110 ST,115375
111 ST,149063


167229179
